###### **1º Passo: Local salvar os dados**

In [1]:
raw_dados_publicos_abfss_files = "abfss://3ed9004c-a27b-47fb-8a19-3a0a8508fbf3@onelake.dfs.fabric.microsoft.com/34b49a7f-cf79-410e-8c7a-2b00f7d44771/Files"

StatementMeta(, 9e3e75b4-d925-42be-ac0f-de6befa58aeb, 3, Finished, Available, Finished)

###### **2º Passo: tabelas que estão tratadas**

In [2]:
tabelas = [
    "NCM", "NCM_SH", "NCM_CUCI", "NCM_ISIC", "ISIC_CUCI", 
    "NCM_CGCE", "NCM_FAT_AGREG", "NCM_PPE", "NCM_PPI", 
    "NCM_UNIDADE", "PAIS", "PAIS_BLOCO", "UF_MUN", "VIA"
]

StatementMeta(, 9e3e75b4-d925-42be-ac0f-de6befa58aeb, 4, Finished, Available, Finished)

In [3]:
caminho_base_origem = f"{raw_dados_publicos_abfss_files}/{'Mdic_Dimensoes'}"

StatementMeta(, 9e3e75b4-d925-42be-ac0f-de6befa58aeb, 5, Finished, Available, Finished)

###### **3º Passo: Loop para salvar os dados na Silver**

In [ ]:
for tabela in tabelas:
    nome_tabela_final = f"tb_dim_{tabela}"
    
    try:
        # Verifica se a tabela já existe no catálogo do Lakehouse
        if spark.catalog.tableExists(nome_tabela_final):
            print(f"⏭️  Pulando: A tabela '{nome_tabela_final}' já existe.")
            continue
            
        # 1. Lemos o arquivo original
        caminho_leitura = f"{caminho_base_origem}/{tabela}.parquet"
        df = spark.read.format("parquet").load(caminho_leitura)
        
        # 2. Escrevemos como TABELA (Usamos mode("error") ou apenas omitimos o mode, 
        # mas como já checamos antes, o overwrite aqui seria seguro apenas para esta execução)
        df.write.format("delta") \
          .mode("overwrite") \
          .option("overwriteSchema", "true") \
          .saveAsTable(nome_tabela_final)
        
        print(f"✅ Tabela '{nome_tabela_final}' criada com sucesso!")
        
    except Exception as e:
        print(f"❌ Erro ao processar {tabela}: {str(e)[:150]}")



print("\n--- Processo concluído ---")

###### **4º Passo: Dados complementares de geolocalização**

In [ ]:
nome_tabela_final = "tb_dim_municipios_geoloc"

try:
    # verifica se já existe
    if spark.catalog.tableExists(nome_tabela_final):
        print(f"Pulando: A tabela '{nome_tabela_final}' já existe.")

    else:
        !pip install geobr geopandas pyarrow

        import geopandas as gpd
        from geobr import read_municipality

        # carrega municípios
        mun = read_municipality(code_muni="all", year=2024)

        # reprojeta para lat/lon
        mun = mun.to_crs(epsg=4326)

        # centroides
        mun["latitude"] = mun.geometry.centroid.y
        mun["longitude"] = mun.geometry.centroid.x

        # seleciona colunas finais
        df_coords = mun[
            [
                "code_muni",
                "name_muni",
                "abbrev_state",
                "name_state",
                "name_region",
                "latitude",
                "longitude"
            ]
        ]

        # GeoPandas → Pandas → Spark
        df_spark = spark.createDataFrame(
            df_coords.drop(columns="geometry", errors="ignore")
        )

        # salva como Delta
        df_spark.write.format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .saveAsTable(nome_tabela_final)

        print(f"Tabela '{nome_tabela_final}' criada com sucesso!")

except Exception as e:
    print(f"Erro ao salvar geolocalização dos municípios: {str(e)[:200]}")


StatementMeta(, b16bad86-97dc-4e40-94b6-82b40664eb88, 6, Finished, Available, Finished)

Pulando: A tabela 'tb_dim_municipios_geoloc' já existe.
